<a href="https://colab.research.google.com/github/srutisam/Empathy-Bot/blob/main/Empathychatbotpervasive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-cors pyngrok nltk tensorflow


In [ ]:
import nltk
import shutil

# Force delete any broken punkt folder
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)

# Download the required packages again
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Helps with lemmatization


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
nltk.download('punkt_tab')


In [ ]:
import os
import random
import json
import pickle
import nltk
import numpy as np
import tensorflow as tf

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from pyngrok import ngrok,conf

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Load intents
with open('intents.json') as file:
    intents = json.load(file)

# Train or Load Model
if not os.path.exists("chatbot_model.h5"):
    words, classes, documents = [], [], []
    ignore_chars = ['?', '!', '.', ',']

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            word_list = pattern.lower().split()
            words.extend(word_list)
            documents.append((word_list, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_chars]
    words = sorted(list(set(words)))
    classes = sorted(list(set(classes)))

    training = []
    output_empty = [0] * len(classes)

    for doc in documents:
        pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
        bag = [1 if word in pattern_words else 0 for word in words]
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])

    random.shuffle(training)
    train_x = np.array([x[0] for x in training])
    train_y = np.array([x[1] for x in training])

    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y[0]), activation='softmax'))

    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
    model.save("chatbot_model.h5")
    pickle.dump(words, open('words.pkl', 'wb'))
    pickle.dump(classes, open('classes.pkl', 'wb'))
else:
    model = tf.keras.models.load_model("chatbot_model.h5")
    words = pickle.load(open('words.pkl', 'rb'))
    classes = pickle.load(open('classes.pkl', 'rb'))

# Chatbot logic
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    results = [[i, r] for i, r in enumerate(res) if r > 0.25]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{'intent': classes[r[0]], 'probability': str(r[1])} for r in results]
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "I'm sorry, I don't understand."

# Flask App
app = Flask(__name__)
CORS(app)

# Basic HTML UI template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Empathy Chatbot</title>
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            background: linear-gradient(135deg, #89f7fe 0%, #66a6ff 100%);
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }

        .chat-container {
            background-color: white;
            width: 400px;
            max-width: 90%;
            border-radius: 15px;
            box-shadow: 0 8px 20px rgba(0,0,0,0.2);
            padding: 20px;
            display: flex;
            flex-direction: column;
        }

        h2 {
            text-align: center;
            color: #333;
            margin-bottom: 10px;
        }

        #chatbox {
            flex: 1;
            height: 300px;
            overflow-y: auto;
            border: 1px solid #ddd;
            padding: 10px;
            border-radius: 10px;
            margin-bottom: 15px;
            background-color: #f9f9f9;
        }

        #chatbox p {
            margin: 8px 0;
        }

        .user {
            color: #2c3e50;
        }

        .bot {
            color: #27ae60;
        }

        .input-area {
            display: flex;
        }

        #user_input {
            flex: 1;
            padding: 10px;
            border-radius: 8px;
            border: 1px solid #ccc;
            font-size: 14px;
        }

        button {
            padding: 10px 15px;
            margin-left: 8px;
            background-color: #3498db;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-weight: bold;
        }

        button:hover {
            background-color: #2980b9;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <h2>💬 Empathy Chatbot</h2>
        <div id="chatbox"></div>
        <div class="input-area">
            <input type="text" id="user_input" placeholder="Say something..." />
            <button onclick="send()">Send</button>
        </div>
    </div>

    <script>
        async function send() {
            const msg = document.getElementById("user_input").value.trim();
            if (!msg) return;

            document.getElementById("chatbox").innerHTML += "<p class='user'><b>You:</b> " + msg + "</p>";

            const res = await fetch('/get_response', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({message: msg})
            });

            const data = await res.json();
            document.getElementById("chatbox").innerHTML += "<p class='bot'><b>Bot:</b> " + data.response + "</p>";
            document.getElementById("user_input").value = "";

            const chatbox = document.getElementById("chatbox");
            chatbox.scrollTop = chatbox.scrollHeight;
        }
    </script>
</body>
</html>
"""


@app.route('/')
def home():
    return render_template_string(html_template)

@app.route('/get_response', methods=['POST'])
def chatbot_response():
    message = request.json['message']
    ints = predict_class(message)
    response = get_response(ints, intents)
    return jsonify({'response': response})

# Launch Flask with Ngrok tunnel
conf.get_default().auth_token = "2vOyoZStXs3D9iqGaok0FXVHPxf_3y7UNXmUQUMsUcNFSJjTC"
public_url = ngrok.connect(5000)
print("Chatbot running on:", public_url)
app.run(port=5000)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0657 - loss: 2.4993    
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1605 - loss: 2.3678  
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2038 - loss: 2.2893 
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1139 - loss: 2.2861     
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2755 - loss: 2.1943 
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2034 - loss: 2.1607     
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3922 - loss: 2.1141 
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3195 - loss: 2.0030 
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4239 - loss: 1.9067 
Epoch 10/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4253 - loss: 1.7942 
Epoch 11/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5092 - loss: 1.6261 
Epoch 12/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 

Chatbot running on: NgrokTunnel: "https://4155-35-247-50-114.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 06:51:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 06:51:57] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


...

INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 06:52:45] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 06:53:04] "POST /get_response HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [08/Apr/2025 06:53:25] "POST /get_response HTTP/1.1" 200 -
